In [ ]:
import os
import shutil

import pandas as pd
import numpy as np

from os import path
from glob import glob
from datetime import datetime

In [ ]:
archive_dir = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/archive/'

In [ ]:
dirs = sorted(glob(archive_dir + '*'))

In [ ]:
date_range = pd.date_range(datetime(2019, 10, 1, 0), datetime(2020, 5, 31, 0))

for date in date_range:
    path = archive_dir + date.strftime('%Y%m%d')
    os.makedirs(path, exist_ok=True)

In [ ]:
verbose = False

with open('./archive_assessment.csv', 'w') as wfp:

    wfp.write('%s,%s,%s,%s,%s,%s,%s,%s\n'%(
        'model', 'date', 'init', 'model_grib', 'model_idx', 
        'model_subdirs', 'model_nc', 'model_other'))

    for date_dir in dirs:

        date = date_dir.split('/')[-1]
        model_dirs = sorted(glob(date_dir + '/models/*'))

        for model_dir in model_dirs:

            model = model_dir.split('/')[-1]
            model_grib = sorted(glob(model_dir + '/*.grib2'))
            model_idx = sorted(glob(model_dir + '/*.idx'))
            [os.remove(f) for f in model_idx]

            model_subdirs = sorted([f for f in glob(model_dir + '/*') 
                            if ((f not in model_grib) & (f not in model_idx))])

            subdirs = [sd.split('/')[-1] for sd in model_subdirs]

            # Break this out to determine particular file counts for each init when needed
            model_nc, model_other = [], []
            for model_subdir in model_subdirs:

                subdir = model_subdir.split('/')[-1]
                model_nc = sorted(glob(model_subdir + '/*.nc'))
                model_other = sorted([f for f in glob(model_subdir + '/*') if f not in model_nc])
                
            if verbose:
                print(model_dir)
                print('\tn grib: {}\n\tn idx: {}\n\tn subdirs: {}\n\tsubdirs: {}\n\tn ncdf: {}\n\tn other: {}\n'.format(
                    len(model_grib), len(model_idx), len(model_subdirs), '; '.join(subdirs), len(model_nc), len(model_other)))

            wfp.write('%s, %s, %s, %d, %d, %d, %d, %d\n'%(
                model, date, '; '.join(subdirs),
                len(model_grib), len(model_idx), len(model_subdirs), len(model_nc), len(model_other)))

In [ ]:
counts = pd.read_csv('./archive_assessment.csv').set_index(['date', 'model'])
counts.head(10)